In [2]:
%load_ext autoreload
%matplotlib inline

In [7]:
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
import xarray as xr

import os,sys,glob
import re

## Add own library to path
workdir = os.getcwd()
repodir = workdir
projectname = 'EUREC4A_rad_profiles'
while os.path.basename(repodir) != projectname:
    repodir = os.path.dirname(repodir)
print('repo:',repodir)
thismodule = sys.modules[__name__]


inputdir = os.path.join(repodir,'input/HALO/20200124/AVAPS_Dropsondes/processed/',)
figdir = os.path.join(repodir,'figures')
moviedir = os.path.join(repodir,'movies')

## Graphical parameters
#plt.style.use(os.path.join(matplotlib.get_configdir(),'stylelib/presentation.mplstyle'))

repo: /run/media/ludo/DATA/google-drive/Thèse/EUREC4a/github/EUREC4A_rad_profiles


In [8]:
inputfiles = glob.glob(os.path.join(inputdir,'D*_PQC.nc'))
inputfiles.sort()

In [9]:
data = []

for file in inputfiles :
    data.append(xr.open_dataset(file).dropna(dim='time',subset=['time']).\
                swap_dims({'time':'pres'}).reset_coords().\
                dropna(dim='pres',subset=['alt','pres','u_wind','v_wind','lat','lon','mr'],how='any'))

Nsondes = len(data)

In [ ]:
# Get min and max
pres_min = np.floor(np.min([data[i].pres.min() for i in range(Nsondes)]))
pres_max = np.ceil(np.max([data[i].pres.max() for i in range(Nsondes)]))

print('min,max = ',pres_min, pres_max)

In [ ]:
# interpolate on same grid
data_interp = []

for i in range(Nsondes) :
    data_interp.append(data[i].interp(pres=np.arange(pres_min,pres_max,1)))

In [ ]:
combined = xr.concat(data_interp[:],dim='sonde_no')

In [ ]:
ax = plt.plot(combined.mr.mean(dim='sonde_no'),combined.pres)
ax.revert('y')
plt.show()